# Clase práctica III: Ejercicios de Aplicación

## Conceptos previos

- Speedup: Medida del rendimiento relativo entre dos arquitecturas o estrategias de paralelización para ejecutar un mismo problema

$$Speedup = \frac{T_{base}}{T_{optimizado}}$$

- SIMD (Single Instruction Multiple Data) o Vectorización: Ejecución de una misma instrucción sobre distintos datos en varias unidades de procesamiento

## Ejercicio 1: Calcular el Speedup

### A. Calcular el tiempo de ejecución base ($T_{base}$)

- El siguiente ejercicio presenta una suma y multiplicación de dos vectores (`v1` y `v2`) producidos de forma aleatoria y definidos como punto flotante en las líneas 11-21. 
- El tamaño de los vectores (`vsize`) es 1000 y vamos a repetir la multiplicación 200 veces (`repetitions`). 
- Guardamos el resultado de la suma y la multiplicación en los vectores `v3` y `v4` que son primero producidos en las líneas 25-27.
- La suma y multiplicación de los vectores `v1` y `v2` se ejecuta en las líneas 33-35.
- En las líneas 37-38 visualizamos el valor de los vectores `v3` y `v4` mediante el comando `std::cout`.

1. **Paso 1**: Creamos el archivo vector.cpp

In [41]:
%%writefile vector.cpp
#include <cstdio>
#include <vector>
#include <stdlib.h>
#include <iostream>

int main(){

        int repetitions = 200;
        int vsize = 1000;

        std::vector<float> v1(vsize);
        std::vector<float> v2(vsize);

        std::cout << "Entrada: \n";

        for (size_t i = 0; i < vsize; i++) {
                v1[i] = static_cast<float>((rand() % vsize)) / vsize;
                v2[i] = static_cast<float>((rand() % vsize)) / vsize;
                std::cout << "v1[" << i << "] = " << v1[i] << "\n";
                std::cout << "v2[" << i << "] = " << v2[i] << "\n";
        }

//      #pragma omp parallel for
//      #pragma omp parallel      
        for(size_t j = 0; j < repetitions; j++){
                std::vector<float> v3(vsize);
                std::vector<float> v4(vsize);

                std::cout << "Repeticion #" << j << "\n";

                // Add and multiply random vectors
//              #pragma omp simd
                for(size_t i = 0; i < vsize; i++){
                      v3[i] = v1[i] + v2[i];
                      v4[i] = v1[i] * v2[i];
                      std::cout << "v3[" << i << "] = " << v3[i] << "\n";
                      std::cout << "v4[" << i << "] = " << v4[i] << "\n";
                }
        }

        return 0;
}

Overwriting vector.cpp


2. **Paso 2**: Creamos el archivo compilador

In [24]:
%%writefile compi_vector.pbs
#PBS -S /bin/bash
#PBS -N compi_base
#PBS -l nodes=1,walltime=00:10:00
dpcpp -o /home/u196481/CPAR-INTRO/compi_base /home/u196481/CPAR-INTRO/vector.cpp
/home/u196481/CPAR-INTRO/compi_base

Overwriting compi_vector.pbs


3. **Paso 3**: Ejecutamos el job, comprobamos si está corriendo y vemos el contenido del archivo _o.xxxx_ cuando haya terminado de ejecutar

In [25]:
!qsub compi_vector.pbs

2345342.v-qsvr-1.aidevcloud


In [30]:
!qstat

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2345330.v-qsvr-1           ...ub-singleuser u196481         00:00:25 R jupyterhub     
2345342.v-qsvr-1           compi_base       u196481                0 Q batch          


In [31]:
!qdel 2345342.v-qsvr-1.aidevcloud

3. **Paso alternativo**: En caso el job demore mucho en ejecutar, podemos hacer uso de nuestro nodo ya habilitado por DevCloud al momento de abrir Jupyter Notebook. Para ello usamos la función siguiente directamente en el terminal:

`dpcpp -o /home/u196481/CPAR-INTRO/compi_base /home/u196481/CPAR-INTRO/vector.cpp`

En el terminal, vamos a observar algo similar al output en el archivo de extensión _o.xxxx_ luego de subir un job:

![alt text](vector_1.png)


4. **Paso 4**: Para ver el tiempo base, es decir el tiempo que demora el algoritmo sin ningún tipo de paralelización, usamos el comando siguiente en el terminal:

`time ./compi_base`

Finalmente guardamos el tiempo de usuario: *User time = 0.424s*

![alt text](vector_2a.png)

### B. Calcular el tiempo optimizado 1 ($T^1_{optimizado}$)

- El siguiente ejercicio presenta el mismo código que `vector.cpp` con la diferencia que estamos activando el la opción `#pragma omp parallel for` (paralelización por _threads_ o hilos) en la línea 23. 
- El objetivo es ver el incremento del rendimiento debido a una paralelización por threads para este ejemplo simple. 
- Para ello, tenemos que ejecutar los mismos pasos anteriores

In [39]:
%%writefile vector2.cpp
#include <cstdio>
#include <vector>
#include <stdlib.h>
#include <iostream>

int main(){

        int repetitions = 200;
        int vsize = 1000;

        std::vector<float> v1(vsize);
        std::vector<float> v2(vsize);

        std::cout << "Entrada: \n";

        for (size_t i = 0; i < vsize; i++) {
                v1[i] = static_cast<float>((rand() % vsize)) / vsize;
                v2[i] = static_cast<float>((rand() % vsize)) / vsize;
                std::cout << "v1[" << i << "] = " << v1[i] << "\n";
                std::cout << "v2[" << i << "] = " << v2[i] << "\n";
        }

        #pragma omp parallel for
//      #pragma omp parallel      
        for(size_t j = 0; j < repetitions; j++){
                std::vector<float> v3(vsize);
                std::vector<float> v4(vsize);

                std::cout << "Repeticion #" << j << "\n";

                // Add and multiply random vectors
//              #pragma omp simd
                for(size_t i = 0; i < vsize; i++){
                      v3[i] = v1[i] + v2[i];
                      v4[i] = v1[i] * v2[i];
                      std::cout << "v3[" << i << "] = " << v3[i] << "\n";
                      std::cout << "v4[" << i << "] = " << v4[i] << "\n";
                }
        }

        return 0;
}

Overwriting vector2.cpp


#### Ejecutar la función:

`dpcpp -o /home/u196481/CPAR-INTRO/compi_T1 /home/u196481/CPAR-INTRO/vector2.cpp`

y luego ejecutar el siguiente programa y guardar el tiempo de usuario: 0.295

`time ./compi_T1`

![alt text](vector_3a.png)

### C. Calcular el tiempo optimizado 2 ($T^2_{optimizado}$)

- El siguiente ejercicio presenta el mismo código que `vector.cpp` con la diferencia que estamos activando el la opción `#pragma omp simd` (paralelización por instrucciones SIMD) en la línea 32. 
- El objetivo es ver el incremento del rendimiento debido a una paralelización por instrucciones SIMD. 
- Para ello, tenemos que ejecutar los mismos pasos anteriores

In [40]:
%%writefile vector3.cpp
#include <cstdio>
#include <vector>
#include <stdlib.h>
#include <iostream>

int main(){

        int repetitions = 200;
        int vsize = 1000;

        std::vector<float> v1(vsize);
        std::vector<float> v2(vsize);

        std::cout << "Entrada: \n";

        for (size_t i = 0; i < vsize; i++) {
                v1[i] = static_cast<float>((rand() % vsize)) / vsize;
                v2[i] = static_cast<float>((rand() % vsize)) / vsize;
                std::cout << "v1[" << i << "] = " << v1[i] << "\n";
                std::cout << "v2[" << i << "] = " << v2[i] << "\n";
        }

//      #pragma omp parallel for
//      #pragma omp parallel      
        for(size_t j = 0; j < repetitions; j++){
                std::vector<float> v3(vsize);
                std::vector<float> v4(vsize);

                std::cout << "Repeticion #" << j << "\n";

                // Add and multiply random vectors
                #pragma omp simd
                for(size_t i = 0; i < vsize; i++){
                      v3[i] = v1[i] + v2[i];
                      v4[i] = v1[i] * v2[i];
                      std::cout << "v3[" << i << "] = " << v3[i] << "\n";
                      std::cout << "v4[" << i << "] = " << v4[i] << "\n";
                }
        }

        return 0;
}

Overwriting vector3.cpp


#### Ejecutar la función:

`dpcpp -o /home/u196481/CPAR-INTRO/compi_T2 /home/u196481/CPAR-INTRO/vector3.cpp`

y luego ejecutar el siguiente programa y guardar el tiempo de usuario: 0.021

`time ./compi_T2`

![alt text](vector_4a.png)

### D. Calcular el Speedup1 y Speedup2 

$Speedup1 = \frac{T_{base}}{T^1_{optimizado}}$ = $\frac{0.424}{0.295}$ = $1.43$

$Speedup2 = \frac{T_{base}}{T^2_{optimizado}}$ = $\frac{0.424}{0.353}$ = $1.20$

- Ambas alternativas ofrecen una mejora en la performancia de nuestro algoritmo. 
- En cursos posteriores nosotros veremos otras herramientas *Vtune* y *Advisor* que nos pueden ayudar a optimizar nuestros algoritmos de manera mucho más eficiente. 